In [2]:
from pathlib import Path
import re
res_dir = Path("res")

# recursive
idf_files = sorted(p for p in res_dir.rglob("*.idf") if p.is_file())
# non-recursive
# idf_files = sorted(p for p in res_dir.glob("*.idf") if p.is_file())
def detect_idf_version(file_path):
    """
    Detect the EnergyPlus version from an IDF file.
    Returns a version string like '8.0' or '24.1.0', or None if not found.
    """
    file_path = Path(file_path)
    text = file_path.read_text(encoding="utf-8", errors="ignore")

    # Regex captures e.g.:
    # Version, 8.0; !- Version Identifier
    # Version,8.0; !-version
    # Version , 9.6.0;
    pattern = re.compile(
        r"(?i)\bVersion\s*,\s*([\d\.]+)\s*;?"
    )

    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    return None
idf_versions = []
for p in idf_files:
    version = detect_idf_version(p)
    idf_versions.append(version)

In [8]:
import os
import re
import shutil
import subprocess
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

# Directory where the Transition executables are located
TRANSITION_DIR = Path(r"C:\EnergyPlusV24-1-0\PreProcess\IDFVersionUpdater")
OUTPUT_DIR = Path("out")
LOG_DIR = Path("logs")
OUTPUT_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)

# Pattern for valid transition executables
pattern = re.compile(
    r"Transition-V(\d+)-(\d+)-(\d+)-to-V(\d+)-(\d+)-(\d+)\.exe", re.IGNORECASE
)

# Create timestamped log file
LOG_FILE = LOG_DIR / f"update_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

def log(msg):
    """Write log entry to file with timestamp."""
    timestamp = datetime.now().strftime("[%H:%M:%S]")
    text = f"{timestamp} {msg}\n"
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(text)

def parse_version(parts):
    return tuple(int(p) for p in parts)

def find_transitions():
    """Return sorted list of (exe_path, from_version, to_version)."""
    transitions = []
    for exe in TRANSITION_DIR.glob("Transition-V*-to-V*.exe"):
        m = pattern.search(exe.name)
        if m:
            from_v = parse_version(m.groups()[0:3])
            to_v   = parse_version(m.groups()[3:6])
            transitions.append((exe, from_v, to_v))
    transitions.sort(key=lambda x: x[1])
    return transitions

def version_tuple(version_str):
    parts = [int(p) for p in version_str.split('.')]
    while len(parts) < 3:
        parts.append(0)
    return tuple(parts[:3])

def run_transition(exe, file_path):
    """Run one transition executable non-interactively with trailing newline."""
    abs_path = str(Path(file_path).resolve())
    cmd = f'(echo full & echo yes & echo yes & echo {abs_path} & echo.) | "{exe}"'
    result = subprocess.run(
        cmd,
        shell=True,
        cwd=exe.parent,
        capture_output=True,
        text=True
    )
    return result

def auto_update(file_path, version='7.2'):
    file_path = Path(file_path)
    current_ver = version_tuple(version)
    transitions = find_transitions()
    work_file = file_path
    first_update = True

    log(f"=== Starting conversion chain for: {file_path.name} ===")
    log(f"Initial version: {current_ver}")

    # tqdm progress bar for user feedback
    steps = [t for t in transitions if t[1] >= current_ver]
    pbar = tqdm(total=len(steps), desc="Updating", ncols=80, unit="step")

    for exe, from_v, to_v in steps:
        if from_v == current_ver:
            log(f"Running {exe.name} ({from_v} -> {to_v}) on {work_file.name}")
            result = run_transition(exe, work_file)

            log(f"Command executed: {result.args}")
            log(f"Exit code: {result.returncode}")
            if result.stdout.strip():
                log(f"--- STDOUT ---\n{result.stdout.strip()}")
            if result.stderr.strip():
                log(f"--- STDERR ---\n{result.stderr.strip()}")

            new_file = work_file.with_suffix(work_file.suffix + "new")
            if new_file.exists():
                if first_update:
                    updated_file = work_file.with_name(
                        work_file.stem + "_updated" + work_file.suffix
                    )
                    first_update = False
                else:
                    updated_file = work_file  # overwrite existing file

                shutil.move(new_file, updated_file)
                work_file = updated_file
                current_ver = to_v
                log(f"✅ Successfully converted to version {to_v}, file now: {work_file.name}")
            else:
                log(f"❌ Conversion failed at {exe.name} (no .idfnew file found)")
                break

        pbar.update(1)

    pbar.close()

    final_out = OUTPUT_DIR / work_file.name
    shutil.copy(work_file, final_out)
    log(f"✅ Final updated file saved at: {final_out.resolve()}")
    log(f"=== Conversion chain complete ===")

    print(f"\nLog saved to: {LOG_FILE.resolve()}")


In [24]:
for idf_file, idf_version in zip(idf_files, idf_versions):
    auto_update(idf_file, version=idf_version)

Updating: 100%|█████████████████████████████████| 4/4 [01:46<00:00, 26.65s/step]



Log saved to: E:\Coding\IDFUpdater\logs\update_log_20251110_030443.txt


Updating: 100%|███████████████████████████████| 21/21 [02:35<00:00,  7.39s/step]



Log saved to: E:\Coding\IDFUpdater\logs\update_log_20251110_030443.txt


Updating: 100%|███████████████████████████████| 21/21 [02:35<00:00,  7.40s/step]


...


Log saved to: E:\Coding\IDFUpdater\logs\update_log_20251110_030443.txt


Updating: 100%|█████████████████████████████████| 2/2 [00:24<00:00, 12.49s/step]



Log saved to: E:\Coding\IDFUpdater\logs\update_log_20251110_030443.txt


Updating: 100%|█████████████████████████████████| 2/2 [00:24<00:00, 12.48s/step]


Log saved to: E:\Coding\IDFUpdater\logs\update_log_20251110_030443.txt


## convert to epjson

"C:\EnergyPlusV24-1-0\energyplus.exe" --convert-only --output-directory "E:\Coding\IDFUpdater\epjson" "E:\Coding\IDFUpdater\out\ASHRAE901_ApartmentMidRise_STD2022_Rochester_updated.idf"

In [ ]:
import json
epjson_path = Path("epjson/ASHRAE901_ApartmentMidRise_STD2022_Rochester_updated.epJSON")

with epjson_path.open("r", encoding="utf-8") as f:
    epjson_data = json.load(f)

print(f"Loaded EPJSON: {epjson_path} ({len(epjson_data)} top-level keys)")

Loaded EPJSON: epjson\US+MF+CZ6A+gasfurnace+crawlspace+IECC_2024_updated.epJSON (104 top-level keys)


In [46]:
epjson_data['Zone']

{'Breezeway': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'attic': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': -41.122974,
  'y_origin': 15.731187,
  'z_origin': 0.0},
 'crawlspace': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'living_unit1_BackRow_BottomFloor': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'living_unit1_BackRow_MiddleFloor': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'living_unit1_BackRow_TopFloor': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'living_unit1_FrontRow_BottomFloor': {'direction_of_relative_north': 0.0,
  'multiplier': 1,
  'x_origin': 0.0,
  'y_origin': 0.0,
  'z_origin': 0.0},
 'li

In [5]:
list(epjson_data.keys())

['AirLoopHVAC',
 'AirLoopHVAC:ControllerList',
 'AirLoopHVAC:OutdoorAirSystem',
 'AirLoopHVAC:OutdoorAirSystem:EquipmentList',
 'AirLoopHVAC:ReturnPath',
 'AirLoopHVAC:SupplyPath',
 'AirLoopHVAC:UnitarySystem',
 'AirLoopHVAC:ZoneMixer',
 'AirLoopHVAC:ZoneSplitter',
 'AirTerminal:SingleDuct:ConstantVolume:NoReheat',
 'AvailabilityManager:Scheduled',
 'AvailabilityManagerAssignmentList',
 'Branch',
 'BranchList',
 'Building',
 'BuildingSurface:Detailed',
 'Coil:Cooling:DX:SingleSpeed',
 'Coil:Heating:Fuel',
 'Construction',
 'Construction:FfactorGroundFloor',
 'Controller:MechanicalVentilation',
 'Controller:OutdoorAir',
 'ConvergenceLimits',
 'Curve:Biquadratic',
 'Curve:Quadratic',
 'DesignSpecification:OutdoorAir',
 'DesignSpecification:ZoneAirDistribution',
 'ElectricEquipment',
 'ElectricLoadCenter:Distribution',
 'ElectricLoadCenter:Generators',
 'ElectricLoadCenter:Inverter:PVWatts',
 'EnergyManagementSystem:Actuator',
 'EnergyManagementSystem:GlobalVariable',
 'EnergyManagementSy

!-   ===========  ALL OBJECTS IN CLASS: OUTPUT:VARIABLE ===========

Output:Variable,
    *,                       !- Key Value
    Electric Equipment Electricity Rate ,  !- Variable Name
    Hourly;                  !- Reporting Frequency

Output:Variable,
    *,                       !- Key Value
    Lights Electricity Rate ,!- Variable Name
    Hourly;                  !- Reporting Frequency

Output:Variable,
    *,                       !- Key Value
    Exterior Lights Electricity Rate ,  !- Variable Name
    Hourly;                  !- Reporting Frequency

"C:\EnergyPlusV24-1-0\energyplus.exe" -r -x -w "E:\Coding\IthacaSmartMeter\tests\energy_atlas\data\wea.epw" -d "E:\Coding\IDFUpdater\sims" "E:\Coding\IDFUpdater\sims\ASHRAE901_ApartmentMidRise_STD2022_Rochester_updated.idf"


## Read from EPJson

In [57]:
import json
from pathlib import Path
from collections import defaultdict
from pprint import pprint

def load_epjson(path):
    with open(path, 'r') as f:
        return json.load(f)

def polygon_area_xy(vertices):
    """Compute the 2D area (projected on XY) of a polygon."""
    x = [v["vertex_x_coordinate"] for v in vertices]
    y = [v["vertex_y_coordinate"] for v in vertices]
    n = len(vertices)
    area = 0.5 * abs(sum(x[i] * y[(i + 1) % n] - x[(i + 1) % n] * y[i] for i in range(n)))
    return area

def compute_zone_floor_areas(epj):
    surfaces = epj.get("BuildingSurface:Detailed", {})
    zone_areas = {}

    for name, surf in surfaces.items():
        if surf.get("surface_type", "").lower() == "floor":
            zone = surf.get("zone_name")
            verts = surf.get("vertices", [])
            if not zone or len(verts) < 3:
                continue
            area = polygon_area_xy(verts)
            zone_areas[zone] = zone_areas.get(zone, 0.0) + area

    return zone_areas


def extract_object_schedules(epj, obj_type, zone_key="zone_or_zonelist_name"):
    """Generic extractor for equipment-like objects with schedule reference."""
    objs = epj.get(obj_type, {})
    out = defaultdict(list)
    for name, o in objs.items():
        zone = o.get(zone_key)
        sched = o.get("schedule_name")
        level = (
            o.get("design_level")
            or o.get("lighting_level")
            or o.get("peak_flow_rate")
            or None
        )
        if zone and sched:
            out[zone].append({"name": name, "schedule": sched, "level": level})
    return out

def get_schedule_day(epj, sched_name):
    """Return 24-hour list of values for a given schedule name."""
    # Handle Schedule:Day:Hourly or Schedule:Compact
    if "Schedule:Day:Hourly" in epj and sched_name in epj["Schedule:Day:Hourly"]:
        vals = epj["Schedule:Day:Hourly"][sched_name]["hourly_value_field"]
        return [float(v) for v in vals]
    if "Schedule:Compact" in epj and sched_name in epj["Schedule:Compact"]:
        fields = epj["Schedule:Compact"][sched_name]
        # Collect all numeric values following 'Until'
        vals = [float(v) for k, v in fields.items() if isinstance(v, (int, float, str)) and str(v).replace('.','',1).isdigit()]
        # pad/trim to 24 values if necessary
        if len(vals) < 24:
            vals += [vals[-1]]*(24-len(vals))
        return vals[:24]
    return [0.0]*24

def get_annual_day_indices(epj, sched_name):
    """Extract mapping from day types to day schedules from Schedule:Year."""
    year_scheds = epj.get("Schedule:Year", {})
    for name, s in year_scheds.items():
        if s.get("schedule_type_limits_name") == sched_name or s.get("name") == sched_name:
            # simplified: return mapping of month/day ranges to day schedules
            mapping = []
            for k, v in s.items():
                if k.startswith("schedule_day_name"):
                    mapping.append(v)
            return mapping
    return []

def extract_schedules(epj, schedule_names):
    """Build a dictionary of schedule_name -> (daily 24h pattern, annual day index list)"""
    result = {}
    for name in schedule_names:
        day = get_schedule_day(epj, name)
        yearmap = get_annual_day_indices(epj, name)
        result[name] = {"daily": day, "annual_day_indices": yearmap}
    return result

epjson_path = Path("epjson/US+MF+CZ6A+gasfurnace+crawlspace+IECC_2024_updated.epJSON")
epj = load_epjson(epjson_path)
zones = compute_zone_floor_areas(epj)
equip = extract_object_schedules(epj, "ElectricEquipment")
lights = extract_object_schedules(epj, "Lights")
water = extract_object_schedules(epj, "WaterUse:Equipment", zone_key="zone_name")

# Gather all schedule names referenced
all_scheds = set()
for d in [equip, lights, water]:
    for lst in d.values():
        for obj in lst:
            all_scheds.add(obj["schedule"])

schedules = extract_schedules(epj, all_scheds)

result = {}
for z in zones:
    result[z] = {
        "area": zones[z],
        "electric_equipment": equip.get(z, []),
        "lighting": lights.get(z, []),
        "hot_water": water.get(z, []),
    }
    # Attach schedule definitions
    for objlist in [result[z]["electric_equipment"], result[z]["lighting"], result[z]["hot_water"]]:
        for o in objlist:
            o["schedule_data"] = schedules.get(o["schedule"], {})

#pprint(result)





## Scr

In [58]:
list(epj.keys())

['AirLoopHVAC',
 'AirLoopHVAC:ReturnPath',
 'AirLoopHVAC:SupplyPath',
 'AirLoopHVAC:UnitaryHeatCool',
 'AirLoopHVAC:ZoneMixer',
 'AirLoopHVAC:ZoneSplitter',
 'AirTerminal:SingleDuct:ConstantVolume:NoReheat',
 'AvailabilityManager:Scheduled',
 'AvailabilityManagerAssignmentList',
 'Branch',
 'BranchList',
 'Building',
 'BuildingSurface:Detailed',
 'Coil:Cooling:DX:SingleSpeed',
 'Coil:Heating:Fuel',
 'Connector:Mixer',
 'Connector:Splitter',
 'ConnectorList',
 'Construction',
 'Curve:Biquadratic',
 'Curve:Cubic',
 'Curve:Exponent',
 'Curve:Quadratic',
 'DesignSpecification:OutdoorAir',
 'Door',
 'ElectricEquipment',
 'Exterior:Lights',
 'Fan:OnOff',
 'GasEquipment',
 'GlobalGeometryRules',
 'GroundHeatTransfer:Basement:BldgData',
 'GroundHeatTransfer:Basement:ComBldg',
 'GroundHeatTransfer:Basement:EquivAutoGrid',
 'GroundHeatTransfer:Basement:EquivSlab',
 'GroundHeatTransfer:Basement:Insulation',
 'GroundHeatTransfer:Basement:Interior',
 'GroundHeatTransfer:Basement:MatlProps',
 'Groun

In [53]:
epj['ZoneList']

KeyError: 'ZoneList'

In [68]:
epj['Schedule:Week:Compact']

{'BathsWeek': {'data': [{'daytype_list': 'For: Weekdays',
    'schedule_day_name': 'BathsWeekday'},
   {'daytype_list': 'For: CustomDay1', 'schedule_day_name': 'BathsVacation'},
   {'daytype_list': 'For: AllOtherDays',
    'schedule_day_name': 'BathsWeekend'}]},
 'ClothesDryerWeek': {'data': [{'daytype_list': 'For: Weekdays',
    'schedule_day_name': 'ClothesDryerWeekday'},
   {'daytype_list': 'For: CustomDay1',
    'schedule_day_name': 'ClothesDryerVacation'},
   {'daytype_list': 'For: AllOtherDays',
    'schedule_day_name': 'ClothesDryerWeekend'}]},
 'ClothesWasherWeek_equip_sch': {'data': [{'daytype_list': 'For: Weekdays',
    'schedule_day_name': 'ClothesWasherWeekday_equip_sch'},
   {'daytype_list': 'For: CustomDay1',
    'schedule_day_name': 'ClothesWasherVacation_equip_sch'},
   {'daytype_list': 'For: AllOtherDays',
    'schedule_day_name': 'ClothesWasherWeekend_equip_sch'}]},
 'ClothesWasherWeek_flow_sch': {'data': [{'daytype_list': 'For: Weekdays',
    'schedule_day_name': 'Cl

In [65]:
epj['Schedule:Compact']

{'BA_bath_sch': {'data': [{'field': 'Through: 12/31'},
   {'field': 'For: AllDays'},
   {'field': 'Until: 1:00'},
   {'field': 7.77272727272727e-05},
   {'field': 'Until: 2:00'},
   {'field': 3.88636363636364e-05},
   {'field': 'Until: 3:00'},
   {'field': 3.88636363636364e-05},
   {'field': 'Until: 4:00'},
   {'field': 3.88636363636364e-05},
   {'field': 'Until: 5:00'},
   {'field': 7.77272727272727e-05},
   {'field': 'Until: 6:00'},
   {'field': 0.000184602272727273},
   {'field': 'Until: 7:00'},
   {'field': 0.000446931818181818},
   {'field': 'Until: 8:00'},
   {'field': 0.000563522727272727},
   {'field': 'Until: 9:00'},
   {'field': 0.00064125},
   {'field': 'Until: 10:00'},
   {'field': 0.000563522727272727},
   {'field': 'Until: 11:00'},
   {'field': 0.000446931818181818},
   {'field': 'Until: 12:00'},
   {'field': 0.000340056818181818},
   {'field': 'Until: 13:00'},
   {'field': 0.000301193181818182},
   {'field': 'Until: 14:00'},
   {'field': 0.000223465909090909},
   {'field

In [44]:
epj['Schedule:Compact']['ltg_sch_apartment_plugin']

{'data': [{'field': 'Through: 12/31'},
  {'field': 'For: SummerDesignDay'},
  {'field': 'Until: 24:00'},
  {'field': 1},
  {'field': 'For: WinterDesignDay'},
  {'field': 'Until: 24:00'},
  {'field': 0},
  {'field': 'For: Weekdays'},
  {'field': 'Until: 4:00'},
  {'field': 0.010188},
  {'field': 'Until: 5:00'},
  {'field': 0.030555},
  {'field': 'Until: 6:00'},
  {'field': 0.066195},
  {'field': 'Until: 7:00'},
  {'field': 0.071289},
  {'field': 'Until: 8:00'},
  {'field': 0.066195},
  {'field': 'Until: 9:00'},
  {'field': 0.030555},
  {'field': 'Until: 15:00'},
  {'field': 0.020367},
  {'field': 'Until: 16:00'},
  {'field': 0.035649},
  {'field': 'Until: 17:00'},
  {'field': 0.071289},
  {'field': 'Until: 18:00'},
  {'field': 0.101844},
  {'field': 'Until: 19:00'},
  {'field': 0.137493},
  {'field': 'Until: 21:00'},
  {'field': 0.162954},
  {'field': 'Until: 22:00'},
  {'field': 0.112032},
  {'field': 'Until: 23:00'},
  {'field': 0.06111},
  {'field': 'Until: 24:00'},
  {'field': 0.025

In [ ]:
zones = {}
zone_lists = epj['ZoneList']
for name, details in epj['ElectricEquipment'].items():
    schedule_name = details['schedule_name']
    
    
    zones_associated = []
    zone_or_list_name = details['zone_or_zonelist_or_space_or_spacelist_name']
    if zone_or_list_name in zones.keys():
        zones_associated.append(zone_or_list_name)
    elif zone_or_list_name in zone_lists.keys():
        for z in zone_lists[zone_or_list_name]['zones']:
            zones_associated.append(z['zone_name'])
    else:
        raise ValueError(f"Unknown zone or zonelist: {zone_or_list_name}. It probably refers to a space or spacelist, which is not yet supported.")
    
    if details['design_level_calculation_method'] == 'EquipmentLevel':
        p = details['design_level']
        
    elif details['design_level_calculation_method'] == 'Watts/Area':
        ppa = details['watts_per_floor_area']  # * area to be applied later
    else:
        raise ValueError(f"Unknown method: {details['design_level_calculation_method']}")
    print(details['design_level_calculation_method'])
    #['zone_or_zonelist_or_space_or_spacelist_name']

EquipmentLevel
EquipmentLevel
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area
Watts/Area


In [ ]:
epj['ElectricEquipment']

{'Elevators_Lights_Fan': {'design_level': 52.66,
  'design_level_calculation_method': 'EquipmentLevel',
  'end_use_subcategory': 'ElevatorLightsFan',
  'fraction_latent': 0.0,
  'fraction_lost': 0.95,
  'fraction_radiant': 0.0,
  'schedule_name': 'ELEV_LIGHT_FAN_SCH_ADD_DF',
  'zone_or_zonelist_or_space_or_spacelist_name': 'G Corridor'},
 'G Corridor_Elevators_Equip': {'design_level': 4360.9,
  'design_level_calculation_method': 'EquipmentLevel',
  'end_use_subcategory': 'ElevatorLift',
  'fraction_latent': 0.0,
  'fraction_lost': 0.95,
  'fraction_radiant': 0.0,
  'schedule_name': 'BLDG_ELEVATORS',
  'zone_or_zonelist_or_space_or_spacelist_name': 'G Corridor'},
 'Plug_GN1': {'design_level_calculation_method': 'Watts/Area',
  'end_use_subcategory': 'MiscPlug',
  'fraction_latent': 0,
  'fraction_lost': 0,
  'fraction_radiant': 0.5,
  'schedule_name': 'EQP_APT_SCH',
  'watts_per_floor_area': 6.67,
  'zone_or_zonelist_or_space_or_spacelist_name': 'G N1 Apartment'},
 'Plug_GN2': {'design_

In [25]:
epj['BuildingSurface:Detailed']

{'g Ceilin N1A': {'construction_name': 'int_slab_ceiling',
  'number_of_vertices': 4,
  'outside_boundary_condition': 'Surface',
  'outside_boundary_condition_object': 'g Ceilin N1A',
  'sun_exposure': 'NoSun',
  'surface_type': 'Ceiling',
  'vertices': [{'vertex_x_coordinate': 0.0,
    'vertex_y_coordinate': 0.0,
    'vertex_z_coordinate': 3.047851},
   {'vertex_x_coordinate': 0.0,
    'vertex_y_coordinate': 7.619628,
    'vertex_z_coordinate': 3.047851},
   {'vertex_x_coordinate': 11.581835,
    'vertex_y_coordinate': 7.619628,
    'vertex_z_coordinate': 3.047851},
   {'vertex_x_coordinate': 11.581835,
    'vertex_y_coordinate': 0.0,
    'vertex_z_coordinate': 3.047851}],
  'view_factor_to_ground': 'Autocalculate',
  'wind_exposure': 'NoWind',
  'zone_name': 'G N1 Apartment'},
 'g Ceilin N2A': {'construction_name': 'int_slab_ceiling',
  'number_of_vertices': 4,
  'outside_boundary_condition': 'Surface',
  'outside_boundary_condition_object': 'g Ceilin N2A',
  'sun_exposure': 'NoSun',

In [31]:
epj['ZoneList']

{'Mid Floor List': {'zones': [{'zone_name': 'M SW Apartment'},
   {'zone_name': 'M NW Apartment'},
   {'zone_name': 'M SE Apartment'},
   {'zone_name': 'M NE Apartment'},
   {'zone_name': 'M N1 Apartment'},
   {'zone_name': 'M N2 Apartment'},
   {'zone_name': 'M S1 Apartment'},
   {'zone_name': 'M S2 Apartment'},
   {'zone_name': 'M Corridor'}]}}